# 卒業生！

今日は、とてつもなくシンプルなグレードフレームワークを使用してユーザーインターフェイスを構築します。

喜びの準備をしてください！

注：グレーディオ画面は、コンピューターの設定に応じて「ダークモード」または「ライトモード」で表示される場合があります。

In [1]:
# 輸入

import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic

In [2]:
import gradio as gr # そうそう！

In [3]:
# .envというファイルに環境変数をロードします
# キープレフィックスを印刷して、デバッグに役立ちます

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("Openai APIキーが設定されていません")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("人為的APIキーが設定されていません")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google APIキーが設定されていません")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyA5


In [4]:
# Openai、人類、Googleに接続します。あなたがそれらを使用していない場合、クロードまたはグーグルの行にコメントしてください

openai = OpenAI()

claude = anthropic.Anthropic()

google.generativeai.configure()

In [5]:
# 一般的なシステムメッセージ - これ以上snarkyな敵対的なAIはありません！

system_message = "You are a helpful assistant"

In [6]:
# 単純な機能でGPT-4O-MINIへの呼び出しを包みましょう

def message_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    completion = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
    )
    return completion.choices[0].message.content

In [7]:
# これにより、「トレーニングの遮断」、またはトレーニングデータの最新の日付が明らかになります

message_gpt("What is today's date?")

"Today's date is October 10, 2023."

## ユーザーインターフェイス時間！

In [8]:
# これが単純な機能です

def shout(text):
    print(f"Shout has been called with input {text}")
    return text.upper()

In [9]:
shout("hello")

Shout has been called with input hello


'HELLO'

In [10]:
# グラデーションのシンプルな。これは「ライトモード」に表示される可能性があります - これを後でダークモードで作成する方法を示します。

gr.Interface(fn=shout, inputs="textbox", outputs="textbox").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [11]:
# 共有= trueを追加すると、公開的にアクセスできることがわかります
# Huggingfaceのスペースと呼ばれるプラットフォームを使用して、より永続的なホスティングを利用できます。
# 注：一部のウイルス対策ソフトウェアと企業のファイアウォールは、share = trueを使用していることを好まないかもしれません。ワークネットワークで作業している場合は、このテストをスキップすることをお勧めします。

gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(share=True)

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://c1f6ab5bdc2722c539.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [12]:
# Inbrowser = trueを追加すると、新しいブラウザウィンドウが自動的に開きます

gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


## ダークモードの強制

グレードは、ブラウザとコンピューターの設定に応じて、ライトモードまたはダークモードで表示されます。 Gradioをダークモードに強制する方法はありますが、Gradioはユーザーの好みである必要があるため、これに対して推奨しています（特にアクセシビリティの理由で）。ただし、画面にダークモードを強制したい場合は、以下の方法です。

In [13]:
# この変数を定義し、インターフェイスを作成するときにjs = force_dark_modeを渡す

force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never", js=force_dark_mode).launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


In [15]:
# 入力と出力

view = gr.Interface(
    fn=shout,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Textbox(label="Response:", lines=8)],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


In [16]:
# そして今 - 関数を「叫び」から「message_gpt」に変更する

view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Textbox(label="Response:", lines=8)],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


In [17]:
# マークダウンを使用しましょう
# その下のコードで言及されていない場合、System_messageを設定することがなぜ違いをもたらすのか疑問に思っていますか？
# Message_gpt関数に戻って使用されているグローバル変数であるSystem_messageを利用しています（見てください）
# 優れたソフトウェアエンジニアリングの実践ではありませんが、Jupyter Lab R＆Dの間は非常に一般的です！

system_message = "You are a helpful assistant that responds in markdown"

view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.


In [18]:
# 結果をストリーミングするコールを作成しましょう
# ジェネレーターの復習（「収量」キーワード）が必要な場合は、
# Week1フォルダーの中間Pythonノートブックをご覧ください。

def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [19]:
view = gr.Interface(
    fn=stream_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7868
* To create a public link, set `share=True` in `launch()`.


In [20]:
def stream_claude(prompt):
    result = claude.messages.stream(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0.7,
        system=system_message,
        messages=[
            {"role": "user", "content": prompt},
        ],
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

In [21]:
view = gr.Interface(
    fn=stream_claude,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7869
* To create a public link, set `share=True` in `launch()`.


## マイナーな改善

このコードを少し改善しました。

以前は、これらの行がありました。

```
for chunk in result:
  yield chunk
```

実際にこれを達成するためのよりエレガントな方法があります（Pythonの人々はもっと「Pythonic」と呼ぶかもしれません）：

「結果からの利回り」

これについては、Week1フォルダーの中間Pythonノートブックで詳細に説明します。詳細をご覧ください。

In [22]:
def stream_model(prompt, model):
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [23]:
view = gr.Interface(
    fn=stream_model,
    inputs=[gr.Textbox(label="Your message:"), gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7870
* To create a public link, set `share=True` in `launch()`.


# 会社のパンフレットジェネレーターの構築

今、あなたは方法を知っています - それは簡単です！

<テーブルスタイル= "マージン：0;テキストアライグ：左;">
    <tr>
        <td style = "width：150px; height：150px; vertical-align：middle;">
            <img src = "../ fality.jpg" width = "150" height = "150" style = "display：block;" />
        </td>
        <td>
            <h2 style = "color：＃900;">次のいくつかのセルを読む前に</h2>
            <Span style = "color：＃900;">
                これを自分で行ってみてください - 週1、5日目に会社のパンフレットに戻り、最後にグラデーションUIを追加してください。次に、解決策を見てください。
            </span>
        </td>
    </tr>
</table>

In [24]:
# ウェブページを表すクラス

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [25]:
# ビル・Gに感謝します。ビル・Gは、これの以前のバージョンにバグがあることに気付きました！今修正されました。

system_message = "You are an assistant that analyzes the contents of a company website landing page \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown."

In [28]:
def stream_brochure(company_name, url, model):
    yield ""
    prompt = f"Please generate a company brochure for {company_name}. Here is their landing page:\n"
    prompt += Website(url).get_contents()
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [30]:
view = gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Company name:"),
        gr.Textbox(label="Landing page URL including http:// or https://"),
        gr.Dropdown(["GPT", "Claude"], label="Select model")],
    outputs=[gr.Markdown(label="Brochure:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7873
* To create a public link, set `share=True` in `launch()`.
